In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

# Composite Material Mechanics Calculator
class CompositeAnalyzer:
    def __init__(self):
        self.materials = {
            'Carbon/Epoxy (AS4/3501-6)': {'E1': 142e9, 'E2': 9.7e9, 'nu12': 0.3, 'G12': 6.0e9},
            'Glass/Epoxy (S2/3501-6)': {'E1': 45e9, 'E2': 12e9, 'nu12': 0.28, 'G12': 5.5e9},
            'Kevlar/Epoxy': {'E1': 80e9, 'E2': 5.5e9, 'nu12': 0.34, 'G12': 2.2e9}
        }
        
    def calculate_Q_matrix(self, E1, E2, nu12, G12):
        nu21 = nu12 * E2 / E1
        Q = np.zeros((3,3))
        Q[0,0] = E1 / (1 - nu12*nu21)
        Q[0,1] = nu12*E2 / (1 - nu12*nu21)
        Q[1,0] = Q[0,1]
        Q[1,1] = E2 / (1 - nu12*nu21)
        Q[2,2] = G12
        return Q
    
    def rotate_Q(self, Q, theta):
        theta_rad = np.deg2rad(theta)
        c = np.cos(theta_rad)
        s = np.sin(theta_rad)
        
        T = np.array([
            [c**2, s**2, 2*c*s],
            [s**2, c**2, -2*c*s],
            [-c*s, c*s, c**2 - s**2]
        ])
        
        Tinv = np.array([
            [c**2, s**2, -2*c*s],
            [s**2, c**2, 2*c*s],
            [c*s, -c*s, c**2 - s**2]
        ])
        
        return Tinv @ Q @ T
    
    def calculate_ABD(self, layers):
        A = np.zeros((3,3))
        B = np.zeros((3,3))
        D = np.zeros((3,3))
        
        z = 0
        for layer in layers:
            Qbar = layer['Qbar']
            t = layer['thickness']
            z_top = z + t/2
            z_bot = z - t/2
            
            A += Qbar * t
            B += Qbar * (z_top**2 - z_bot**2) / 2
            D += Qbar * (z_top**3 - z_bot**3) / 3
            
            z += t
            
        return A, B, D

# Create analyzer instance
analyzer = CompositeAnalyzer()

# Widgets
material_dropdown = widgets.Dropdown(
    options=list(analyzer.materials.keys()),
    description='Material:',
    style={'description_width': 'initial'}
)

ply_angle = widgets.FloatSlider(
    value=0,
    min=-90,
    max=90,
    step=5,
    description='Ply Angle (deg):',
    continuous_update=False
)

thickness = widgets.FloatText(
    value=0.125,
    description='Thickness (mm):',
    style={'description_width': 'initial'}
)

add_layer_btn = widgets.Button(description="Add Layer")
clear_layers_btn = widgets.Button(description="Clear Layers")
calculate_btn = widgets.Button(description="Calculate ABD Matrix")
output = widgets.Output()
plot_output = widgets.Output()
results_output = widgets.Output()
layer_table_output = widgets.Output()

layers = []

# UI Layout
input_box = widgets.VBox([
    widgets.HBox([material_dropdown, thickness]),
    ply_angle,
    widgets.HBox([add_layer_btn, clear_layers_btn])
])

tab = widgets.Tab(children=[input_box, plot_output, results_output])
tab.set_title(0, 'Input Parameters')
tab.set_title(1, 'Visualization')
tab.set_title(2, 'Results')

ui = widgets.VBox([
    widgets.Label(value="Composite Laminate Analyzer", style={'font_weight': 'bold'}),
    tab,
    widgets.Label("Layer Stacking Sequence:"),
    layer_table_output,
    calculate_btn,
    output
])

# Event Handlers
def update_layer_table():
    with layer_table_output:
        layer_table_output.clear_output()
        if layers:
            df = pd.DataFrame([(l['material'], l['angle'], f"{l['thickness']:.3f}") 
                              for l in layers],
                             columns=['Material', 'Angle (deg)', 'Thickness (mm)'])
            display(df)
        else:
            display(Markdown("No layers added yet."))

def add_layer_btn_clicked(b):
    material = material_dropdown.value
    angle = ply_angle.value
    t = thickness.value
    
    layers.append({
        'material': material,
        'angle': angle,
        'thickness': t
    })
    
    update_layer_table()

def clear_layers_btn_clicked(b):
    layers.clear()
    update_layer_table()
    results_output.clear_output()
    plot_output.clear_output()

def calculate_btn_clicked(b):
    with output:
        output.clear_output()
        try:
            if not layers:
                raise ValueError("No layers added!")
                
            # Calculate Q matrices for each layer
            processed_layers = []
            for layer in layers:
                props = analyzer.materials[layer['material']]
                Q = analyzer.calculate_Q_matrix(
                    props['E1'], props['E2'],
                    props['nu12'], props['G12']
                )
                Qbar = analyzer.rotate_Q(Q, layer['angle'])
                processed_layers.append({
                    'Qbar': Qbar,
                    'thickness': layer['thickness'] * 1e-3  # Convert mm to m
                })
            
            # Calculate ABD matrix
            A, B, D = analyzer.calculate_ABD(processed_layers)
            
            # Display results
            with results_output:
                results_output.clear_output()
                display(Markdown("### ABD Matrix Results"))
                
                display(Markdown("**A Matrix (N/m)**"))
                display(pd.DataFrame(A, columns=['εx', 'εy', 'γxy'], 
                                     index=['σx', 'σy', 'τxy']))
                
                display(Markdown("**B Matrix (N)**"))
                display(pd.DataFrame(B, columns=['εx', 'εy', 'γxy'], 
                                     index=['σx', 'σy', 'τxy']))
                
                display(Markdown("**D Matrix (N·m)**"))
                display(pd.DataFrame(D, columns=['εx', 'εy', 'γxy'], 
                                     index=['σx', 'σy', 'τxy']))
            
            # Plot through thickness
            with plot_output:
                plot_output.clear_output()
                fig, ax = plt.subplots(1, 2, figsize=(12, 6))
                
                # Thickness profile
                z = 0
                thicknesses = [l['thickness'] * 1e-3 for l in layers]  # Convert mm to m
                total_thickness = sum(thicknesses)
                
                for i, (layer, t) in enumerate(zip(layers, thicknesses)):
                    ax[0].barh(i+1, t, left=z, color='skyblue', edgecolor='black')
                    ax[0].text(z + t/2, i+1, f"{layer['angle']}°", 
                             ha='center', va='center')
                    z += t
                
                ax[0].set_yticks(range(1, len(layers)+1))
                ax[0].set_yticklabels([f"Layer {i+1}" for i in range(len(layers))])
                ax[0].set_xlabel('Thickness (m)')
                ax[0].set_title('Laminate Stacking Sequence')
                
                # Stiffness distribution
                z_points = np.linspace(0, total_thickness, 100)
                Ex = np.zeros_like(z_points)
                
                z_prev = 0
                for i, layer in enumerate(processed_layers):
                    z_start = z_prev
                    z_end = z_start + thicknesses[i]
                    mask = (z_points >= z_start) & (z_points <= z_end)
                    Ex[mask] = layer['Qbar'][0,0] / 1e9
                    z_prev = z_end
                
                ax[1].plot(Ex, z_points)
                ax[1].set_xlabel('Ex (GPa)')
                ax[1].set_ylabel('Thickness (m)')
                ax[1].set_title('Stiffness Distribution')
                plt.tight_layout()
                plt.show()
                
        except Exception as e:
            display(Markdown(f"**Error:** {str(e)}"))

# Assign handlers
add_layer_btn.on_click(add_layer_btn_clicked)
clear_layers_btn.on_click(clear_layers_btn_clicked)
calculate_btn.on_click(calculate_btn_clicked)

# Display instructions
display(Markdown("""
# Composite Laminate Analyzer
**Instructions:**
1. Select material from dropdown
2. Set ply angle and thickness (in mm)
3. Click "Add Layer" to build stacking sequence
4. Click "Calculate ABD Matrix" to analyze
"""))

display(ui)
update_layer_table()  # Initialize the layer table display


# Composite Laminate Analyzer
**Instructions:**
1. Select material from dropdown
2. Set ply angle and thickness (in mm)
3. Click "Add Layer" to build stacking sequence
4. Click "Calculate ABD Matrix" to analyze
